<a href="https://colab.research.google.com/github/google-research/tapas/blob/master/notebooks/sqa_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 Priyal Narang

*   List item
*   List item



Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2020 Priyal Narang.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Running a Tapas fine-tuned checkpoint
---
This notebook shows how to load and make predictions with TAPAS model, which was introduced in the paper: [TAPAS: Weakly Supervised Table Parsing via Pre-training](https://arxiv.org/abs/2004.02349)

# Clone and install the repository


First, let's fetch the code from the github repository and install it

In [2]:
! git clone https://github.com/google-research/tapas.git

Cloning into 'tapas'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 109 (delta 25), reused 103 (delta 19), pack-reused 0
Receiving objects: 100% (109/109), 154.25 KiB | 430.00 KiB/s, done.
Resolving deltas: 100% (25/25), done.


In [3]:
! pip uninstall -y tensorflow
! pip install ./tapas

Uninstalling tensorflow-2.2.0:
  Successfully uninstalled tensorflow-2.2.0
Processing ./tapas
     |████████████████████████████████| 71kB 1.9MB/s 
     |████████████████████████████████| 377.0MB 45kB/s 
     |████████████████████████████████| 983kB 23.1MB/s 
     |████████████████████████████████| 3.2MB 39.5MB/s 
     |████████████████████████████████| 491kB 38.8MB/s 
  Created wheel for tapas: filename=tapas-0.0.1.dev0-cp36-none-any.whl size=153403 sha256=6410033ac2b9fcfdcc9ffd0afd0e5806e6947e1c8ac61b21633388b7693de0a7
  Stored in directory: /tmp/pip-ephem-wheel-cache-tsw4xzf3/wheels/2f/0b/d5/7e7fd15d1eb9839bd9768eaa6af2e0446329927b0f0c351387
  Created wheel for frozendict: filename=frozendict-1.2-cp36-none-any.whl size=3149 sha256=91438a92bcbecb4b4b8f5e91f3ee21e60176c29ec3e697e9ec6eb0155bc12de6
  Stored in directory: /root/.cache/pip/wheels/6c/6c/e9/534386165bd12cf1885582c75eb6d0ffcb321b65c23fe0f834
Successfully built tapas frozendict
  Found existing installation: tensorboard 2.2.1

# Fetch models fom Google Storage

Next we can get pretrained checkpoint from Google Storage. For the sake of speed, this is base sized model trained on [SQA](https://www.microsoft.com/en-us/download/details.aspx?id=54253). Note that best results in the paper were obtained with with a large model, with 24 layers instead of 12.

In [4]:
! gsutil cp gs://tapas_models/2020_04_21/tapas_sqa_base.zip . && unzip tapas_sqa_base.zip

Copying gs://tapas_models/2020_04_21/tapas_sqa_base.zip...
| [1 files][  1.0 GiB/  1.0 GiB]   27.1 MiB/s                                   
Operation completed over 1 objects/1.0 GiB.                                      
Archive:  tapas_sqa_base.zip
   creating: tapas_sqa_base/
  inflating: tapas_sqa_base/model.ckpt.data-00000-of-00001  
  inflating: tapas_sqa_base/model.ckpt.index  
  inflating: tapas_sqa_base/README.txt  
  inflating: tapas_sqa_base/vocab.txt  
  inflating: tapas_sqa_base/bert_config.json  
  inflating: tapas_sqa_base/model.ckpt.meta  


# Imports

In [5]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [0]:
from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils

# Load checkpoint for prediction

Here's the prediction code, which will create and `interaction_pb2.Interaction` protobuf object, which is the datastructure we use to store examples, and then call the prediction script.

In [0]:
os.makedirs('results/sqa/tf_examples', exist_ok=True)
os.makedirs('results/sqa/model', exist_ok=True)
with open('results/sqa/model/checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'tapas_sqa_base/model.ckpt{suffix}', f'results/sqa/model/model.ckpt-0{suffix}')

In [0]:
max_seq_length = 512
vocab_file = "tapas_sqa_base/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, queries):
  table = [list(map(lambda s: s.strip(), row.split("|"))) 
           for row in table_data.split("\n") if row.strip()]
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example("results/sqa/tf_examples/test.tfrecord", examples)
  write_tf_example("results/sqa/tf_examples/random-split-1-dev.tfrecord", [])
  
  ! python tapas/tapas/run_task_main.py \
    --task="SQA" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --init_checkpoint="tapas_sqa_base/model.ckpt" \
    --bert_config_file="tapas_sqa_base/bert_config.json" \
    --mode="predict" 2> error


  results_path = "results/sqa/model/test_sequence.tsv"
  all_coordinates = []
  df = pd.DataFrame(table[1:], columns=table[0])
  display(IPython.display.HTML(df.to_html(index=False)))
  print()
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
      coordinates = prediction_utils.parse_coordinates(row["answer_coordinates"])
      all_coordinates.append(coordinates)
      answers = ', '.join([table[row + 1][col] for row, col in coordinates])
      position = int(row['position'])
      print(">", queries[position])
      print(answers)
  return all_coordinates

# Predictions


In [11]:
#Performs well on simple SELECT Queries not involving any aggregation operation
res1 = predict("""
ID  | CustomerName         | Address               | PostalCode       |Age   | Country
1   | Patrick              | Obere Str. 57         | 12209            | 23   | Germany  
2   | Alfred               | Via dei Fiorentini 66 | 05021            | 50   | Italy      
3   | Priyal               | Dwarka                | 110078           | 22   | India
4   | Siddharth            | Dwarka                | 110078           | 22   | India  
5   | Alex                 | California            | 90201            | 34   | Spain  

""", ["what were the customer names?",
      "how many countries are present?",
      "which customers have the same country?",
      "what is the average age of customers?",
      "which customer has the maximum age?"])

is_built_with_cuda: True
is_gpu_available: True
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Training or predicting ...
Evaluation finished after training step 0.


ID,CustomerName,Address,PostalCode,Age,Country
1,Patrick,Obere Str. 57,12209,23,Germany
2,Alfred,Via dei Fiorentini 66,05021,50,Italy
3,Priyal,Dwarka,110078,22,India
4,Siddharth,Dwarka,110078,22,India
5,Alex,California,90201,34,Spain



> what were the customer names?
Patrick, Siddharth, Priyal, Alex, Alfred
> how many countries are present?
Spain, Italy, Germany, India, India
> which customers have the same country?
Siddharth, Alex, Priyal
> what is the average age of customers?
34
> which customer has the maximum age?
Alex


In [10]:
#Testing the model with more data
res2 = predict("""
ProductID  |	ProductName                                 |	SupplierID	  |  CategoryID  |	Unit                |	Price
1	         |  Chais	                                      |  1	          |  1	         |  10 boxes x 20 bags	|  18
2	         |  Chang	                                      |  1	          |  1           | 	24 - 12 oz bottles	|  19
3	         |  Aniseed Syrup 	                            |  1	          |  2	         |  12 - 550 ml bottles	|  10
4	         |  Chef Anton's Cajun Seasoning                |  2	          |  2	         |  48 - 6 oz jars	    |  22
5	         |  Chef Anton's Gumbo Mix	                    |  2	          |  2	         |  36 boxes	          |  21.35
6	         |  Grandma's Boysenberry Spread                |	 3	          |  2	         |  12 - 8 oz jars	    |  25
7	         |  Uncle Bob's Organic Dried Pears	            |  3	          |  7	         |  12 - 1 lb pkgs.	    |  30
8	         |  Northwoods Cranberry Sauce	                |  3	          |  2	         |  12 - 12 oz jars	    |  40
9	         |  Mishi Kobe Niku	                            |  4	          |  6	         |  18 - 500 g pkgs.	  |  97
10	       |  Ikura	                                      |  4	          |  8	         |  12 - 200 ml jars	  |  31
11	       |  Queso Cabrales	                            |  5	          |  4	         |  1 kg pkg.	          |  21
12	       |  Queso Manchego La Pastora                   |	 5	          |  4	         |  10 - 500 g pkgs.	  |  38
13	       |  Konbu	                                      |  6	          |  8	         |  2 kg box	          |  6
14	       |  Tofu	                                      |  6	          |  7	         |  40 - 100 g pkgs.	  |  23.25
15	       |  Genen Shouyu	                              |  6	          |  2	         |  24 - 250 ml bottles	|  15.5
16	       |  Pavlova	                                    |  7	          |  3	         |  32 - 500 g boxes	  |  17.45
17	       |  Alice Mutton	                              |  7	          |  6	         |  20 - 1 kg tins	    |  39
18	       |  Carnarvon Tigers                            |  7	          |  8	         |  16 kg pkg.	        |  62.5
19	       |  Teatime Chocolate Biscuits	                |  8	          |  3	         |  10 boxes x 12 pieces|  9.2
20	       |  Sir Rodney's Marmalade	                    |  8	          |  3	         |  30 gift boxes	      |  81

""", ["what are the product names?",
      "how many ProductIDs are present?",
      "how many units are present for the product Chais?",
      "what is the highest price?",
      "what is the lowest price of all products?"])

is_built_with_cuda: True
is_gpu_available: True
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Training or predicting ...
Evaluation finished after training step 0.


ProductID,ProductName,SupplierID,CategoryID,Unit,Price
1,Chais,1,1,10 boxes x 20 bags,18
2,Chang,1,1,24 - 12 oz bottles,19
3,Aniseed Syrup,1,2,12 - 550 ml bottles,10
4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22
5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35
6,Grandma's Boysenberry Spread,3,2,12 - 8 oz jars,25
7,Uncle Bob's Organic Dried Pears,3,7,12 - 1 lb pkgs.,30
8,Northwoods Cranberry Sauce,3,2,12 - 12 oz jars,40
9,Mishi Kobe Niku,4,6,18 - 500 g pkgs.,97
10,Ikura,4,8,12 - 200 ml jars,31



> what are the product names?
Chais, Tofu, Konbu, Ikura, Northwoods Cranberry Sauce, Mishi Kobe Niku, Uncle Bob's Organic Dried Pears, Chef Anton's Cajun Seasoning, Carnarvon Tigers, Aniseed Syrup, Pavlova, Alice Mutton, Genen Shouyu, Queso Manchego La Pastora, Queso Cabrales, Grandma's Boysenberry Spread, Sir Rodney's Marmalade, Chef Anton's Gumbo Mix, Chang, Teatime Chocolate Biscuits
> how many ProductIDs are present?
19, 10, 1, 8, 13, 4, 18, 9, 16, 7, 12, 3, 17, 15, 6, 20, 11, 2, 14, 5
> how many units are present for the product Chais?
10 boxes x 20 bags
> what is the highest price?
18
> what is the lowest price of all products?
6
